<a href="https://colab.research.google.com/github/diogocristovao/SPBD_tp1/blob/main/docs/labs/projs/spbd2425_tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas para Processamento de Big Data
## TP2 - Energy Meter Live Monitoring


The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

For all the sensors combined:

1. For the current month and current day, compute the running total energy consumed so far. The values should be updated every 5 minutes.

2. For the current month and current day, compute the running total energy consumed so far, **as a percentage**, **compared to the same periods in February 2024**. The values should be updated every 5 minutes.

For each sensor, separately:

3. For the current month and current day, compute the running total energy consumed so far, as a percentage, **comparing the value of each individual sensor, relative to the same results for all the sensors together (as in #1)**. The values should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

**Note:** For simplicity, it is fine to assume the first reading of each day can be used to start counting how much energy has been consumed so far. There is no need to interpolate/estimate the value of the meters at midnight.




## Requeriments

Solve each question using Structured Spark Streaming.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline

December 6.

Penalty of 0.25 grade points per day late.

Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [ ]:
#@title Install PySpark
!pip install pyspark --quiet

In [1]:
#@title Download Archived February Energy Readings
!wget -q -O /tmp/readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!grep "2024-02" /tmp/readings.csv > february-energy-readings.csv
!head -2 february-energy-readings.csv


2024-02-01 00:00:00;D;2615.0
2024-02-01 00:00:18;C;1098.8


In [9]:
#@title Start the Structured Source
!wget -q -O - https://github.com/smduarte/spbd-2425/raw/main/scripts/json_energy_sender.tgz  | tar xfz - 2> /dev/null

!nohup python json_energy_sender/server.py --filename february-energy-readings.csv --speedup 60 > /dev/null 2> /dev/null &


In [12]:
query.stop()


Note: --speedup 60, means the stream is played 60x faster than realtime. Therefore, 1 second in real time corresponds to 1 minute of stream data.


In [13]:
#@title Sample code to process the structured stream...
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .appName("StructuredWebLogExample") \
    .getOrCreate()


# Extract a sample JSON string to infer schema
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)


# Create DataFrame representing the stream of input
# lines from connection to logsender 7777
try:
  json_lines = spark.readStream.format("socket") \
      .option("host", "localhost") \
      .option("port", 7777) \
      .load()

  # Parse the JSON using the inferred schema
  json_lines = json_lines.withColumn("json_data", from_json(col("value"), inferred_schema)) \
    .select("json_data.*")  # Expand the JSON fields into columns


  query = json_lines \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(lambda df, epoch: df.show(10, False)) \
    .start()

  query.awaitTermination(10)
except Exception as err:
  print(err)
  query.stop()

+----+------+------+
|date|energy|sensor|
+----+------+------+
+----+------+------+

+-------------------+------+------+
|date               |energy|sensor|
+-------------------+------+------+
|2024-02-01 00:00:00|2615.0|D     |
|2024-02-01 00:00:25|650.5 |A     |
|2024-02-01 00:00:18|1098.8|C     |
+-------------------+------+------+

+-------------------+------+------+
|date               |energy|sensor|
+-------------------+------+------+
|2024-02-01 00:00:33|966.7 |J     |
|2024-02-01 00:00:54|1874.0|E     |
|2024-02-01 00:00:42|2145.4|H     |
+-------------------+------+------+

+-------------------+------+------+
|date               |energy|sensor|
+-------------------+------+------+
|2024-02-01 00:01:52|841.2 |K     |
|2024-02-01 00:02:20|927.2 |I     |
|2024-02-01 00:02:00|1874.1|E     |
+-------------------+------+------+

+-------------------+------+------+
|date               |energy|sensor|
+-------------------+------+------+
|2024-02-01 00:02:36|841.3 |K     |
|2024-02-01 

Question 1

In [18]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Initialize Spark session
spark = SparkSession \
    .builder \
    .appName("EnergyMeterRunningTotal") \
    .getOrCreate()

# Infer schema from a sample JSON
sample_json = '{"date": "2024-02-01 00:00:00", "sensor": "D", "energy": 2615.0}'
inferred_schema = schema_of_json(sample_json)

try:
    # Create DataFrame representing the stream of input
    json_lines = spark.readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 7777) \
        .load()

    # Parse the JSON data
    parsed_stream = json_lines.withColumn("json_data", from_json(col("value"), inferred_schema)) \
        .selectExpr("json_data.date as timestamp", "json_data.sensor as sensor_id", "json_data.energy as energy") \
        .withColumn("timestamp", to_timestamp(col("timestamp")))  # Convert timestamp to proper type

    # Filter for the current month (February)
    filtered_stream = parsed_stream.filter(month(col("timestamp")) == 2)

    # Define update function for the accumulated state
    def update_running_total(batch_df, epoch_id, state={}):
        # Calculate total energy in the current batch
        batch_total = batch_df.agg(sum("energy")).collect()[0][0]
        # Update the state with the current batch total
        state["total_energy"] = state.get("total_energy", 0) + (batch_total if batch_total else 0)
        # Print the running total
        print(f"Running Total Energy: {state['total_energy']}")

    # Process the stream in batches and update the running total
    query = filtered_stream.writeStream \
        .foreachBatch(update_running_total) \
        .trigger(processingTime="5 minutes") \
        .start()

    query.awaitTermination()
except Exception as err:
    print(f"Error: {err}")
    if 'query' in locals():
        query.stop()


Running Total Energy: 0


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 